## Local Producer

In this exercise, you'll run a producer on your computer to send some arbitrary files to a topic

In [ ]:
# imports
import pathlib, logging, importlib
from threading import Thread
from openmsitoolbox.logging import OpenMSILogger
from openmsistream import UploadDataFile, DataFileUploadDirectory

In [ ]:
# Configure a logger (only needed when running in a Jupyter notebook like this)
logger = OpenMSILogger("LocalProducer", filelevel=None)
importlib.reload(logging)

In [ ]:
# The name of the topic to work with
TOPIC_NAME = "tutorial_data"

# Paths to the config file and the directory holding the test files
repo_root_dir = pathlib.Path().resolve().parent
CONFIG_FILE_PATH = repo_root_dir / "config_files" / "confluent_cloud_broker.config"
TEST_FILE_DIR = repo_root_dir.parent / "tutorial_files"

### First, let's just call UploadDataFile for each file in the directory

This will start up a producer and send every chunk for each file

In [ ]:
# For every file in the folder
for iuf, upload_file_path in enumerate(TEST_FILE_DIR.rglob("*")):
    # Skip any hidden files (like .DS_Store....)
    if upload_file_path.is_dir() or upload_file_path.name.startswith("."):
        continue
    # Create an UploadDataFile and call the function to upload it to the topic
    upload_file = UploadDataFile(upload_file_path, rootdir=TEST_FILE_DIR, logger=logger)
    upload_file.upload_whole_file(CONFIG_FILE_PATH, TOPIC_NAME)

### Next let's watch for new files in a folder using a DataFileUploadDirectory

You could run this as an interactive program from the command line, and type a command to shut it down when you wanted to, but here we'll run it in a separate thread from this notebook.

In [ ]:
def upload_task(upload_directory, *args, **kwargs):
    """Run "upload_files_as_added" for a given DataFileUploadDirectory, and log a message
    when it gets shut down

    Args:
        upload_directory (DataFileUploadDirectory): the DataFileUploadDirectory to run
        args (list): passed through to "upload_files_as_added"
        kwargs (dict): passed through to "upload_files_as_added"
    """
    # This call to "upload_files_as_added" waits until the program is shut down
    uploaded_filepaths = upload_directory.upload_files_as_added(*args, **kwargs)
    msg = (
        f"The following files were uploaded:\n\t"
    )
    msg += "\n\t".join([str(fp) for fp in uploaded_filepaths])
    upload_directory.logger.info(msg)

In [ ]:
# Create the DataFileUploadDirectory
dfud = DataFileUploadDirectory(TEST_FILE_DIR, CONFIG_FILE_PATH, logger=logger)
# Start running its "upload_files_as_added" function in a separate thread
upload_thread = Thread(
    target=upload_task,
    args=(
        dfud,
        TOPIC_NAME,
    ),
)
upload_thread.start()

#### With the above cell running, any files you move into the watched directory will be uploaded

In [ ]:
# Manually shut down the upload directory (if running from the command line this would
# be like typing "q" in the Terminal window)
dfud.control_command_queue.put("q")
upload_thread.join()